Натренувати класифікатори на датасеті

https://archive.ics.uci.edu/dataset/713/auction+verification

Мають бути 3 варіанти класифікаторів (стекінг, бустінг, беггінг)

Обов'язкові кроки:

первинний аналіз даних (відстуність пропусків, наявність категоріальних фіч, ...)
фича інжиніринг (побудувати  1-2 нові фічі)
масштабування фіч
поділ датасету на тренувальну, валідаційну та тестову частини
тренування базової моделі із дефолтними гіперпараметрами
підбір гіперпараметрів 
оцінка результатів

In [64]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, BaggingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV


In [65]:
data = pd.read_csv("../auction+verification/data.csv")
# Первинний аналіз
# Перевіримо наявність пропусків та типи даних
missing_values = data.isnull().sum()
data_types = data.dtypes

# Базова статистика по всіх числових змінних
basic_stats = data.describe()

missing_values, data_types, basic_stats

(process.b1.capacity    0
 process.b2.capacity    0
 process.b3.capacity    0
 process.b4.capacity    0
 property.price         0
 property.product       0
 property.winner        0
 verification.result    0
 verification.time      0
 dtype: int64,
 process.b1.capacity      int64
 process.b2.capacity      int64
 process.b3.capacity      int64
 process.b4.capacity      int64
 property.price           int64
 property.product         int64
 property.winner          int64
 verification.result       bool
 verification.time      float64
 dtype: object,
        process.b1.capacity  process.b2.capacity  process.b3.capacity  \
 count          2043.000000          2043.000000          2043.000000   
 mean              1.000000             2.093979             1.883994   
 std               0.816696             0.811269             0.320310   
 min               0.000000             0.000000             1.000000   
 25%               0.000000             1.000000             2.000000   
 50%     

Результати первинного аналізу:
Відсутність пропусків: Усі стовпці не містять пропусків, що полегшує подальшу обробку даних.

Типи даних:

Більшість змінних мають тип int64 (цілі числа), включаючи process.b1.capacity, property.price тощо.
Цільова змінна verification.result має тип bool.
verification.time має тип float64 (дробове число).
Базова статистика:

Значення в змінних process.b1.capacity, process.b2.capacity, process.b3.capacity, process.b4.capacity варіюються в обмежених діапазонах.
property.price має середнє значення близько 71.47 і варіюється від 59 до 90.
verification.time показує широкий діапазон часу від 77.9 до 44130.5 секунд, із середнім значенням близько 7336.94 секунд.

Об'єднаємо значення з чотирьох стовпців process.b1.capacity, process.b2.capacity, process.b3.capacity, process.b4.capacity в одну ознаку total_process_capacity.

Створимо ознаку relative_verification_time, яка буде обчислена як відношення verification.time до property.price.

In [66]:
# Створимо нові ознаки
data['total_process_capacity'] = (
    data['process.b1.capacity'] +
    data['process.b2.capacity'] +
    data['process.b3.capacity'] +
    data['process.b4.capacity']
)

data['relative_verification_time'] = data['verification.time'] / data['property.price']

# Переглянемо перші кілька рядків оновленого датасету
data[['total_process_capacity', 'relative_verification_time']].head()

,total_process_capacity,relative_verification_time
0,3,2.768079
1,3,3.404407
2,3,2.625235
3,3,1.841356
4,3,1.424444


In [67]:
# Масштабування ознак
scaler = StandardScaler()
scaled_features = scaler.fit_transform(data[['total_process_capacity', 'relative_verification_time', 'property.price', 'verification.time']])

# Перетворимо результат назад у DataFrame
scaled_data = pd.DataFrame(scaled_features, columns=['total_process_capacity', 'relative_verification_time', 'property.price', 'verification.time'])

data.head()

,process.b1.capacity,process.b2.capacity,process.b3.capacity,process.b4.capacity,property.price,property.product,property.winner,verification.result,verification.time,total_process_capacity,relative_verification_time
0,0,0,2,1,59,1,0,False,163.316667,3,2.768079
1,0,0,2,1,59,2,0,False,200.860000,3,3.404407
2,0,0,2,1,59,4,0,False,154.888889,3,2.625235
3,0,0,2,1,59,6,0,False,108.640000,3,1.841356
4,0,0,2,1,60,1,0,True,85.466667,3,1.424444


In [68]:
# Поділ на тренувальні, валідаційні та тестові дані
X = scaled_data
y = data['verification.result']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [69]:
# Логістична регресія
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)

# Випадковий ліс
rf = RandomForestClassifier()
rf.fit(X_train, y_train)


RandomForestClassifier()

In [70]:
# Градієнтний бустинг
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)

GradientBoostingClassifier()

In [71]:
# Стекінг
estimators = [
    ('lr', LogisticRegression()),
    ('rf', RandomForestClassifier()),
    ('gb', GradientBoostingClassifier())
]

stacking = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
stacking.fit(X_train, y_train)

StackingClassifier(estimators=[('lr', LogisticRegression()),
                               ('rf', RandomForestClassifier()),
                               ('gb', GradientBoostingClassifier())],
                   final_estimator=LogisticRegression())

In [72]:
# Беггінг
bagging = BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=10)
bagging.fit(X_train, y_train)

BaggingClassifier(estimator=DecisionTreeClassifier())

In [73]:
# Підбір гіперпараметрів

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, cv=5)
grid_search.fit(X_train, y_train)

print("Найкращі параметри:", grid_search.best_params_)
best_rf = grid_search.best_estimator_

Найкращі параметри: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}


In [74]:
# Оцінка результатів

models = {
    'Логістична регресія': log_reg,
    'Випадковий ліс': rf,
    'Градієнтний бустинг': gb,
    'Стекінг': stacking,
    'Беггінг': bagging,
    'Найкращий випадковий ліс': best_rf
}

for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"\nРезультати для {name}:")
    print(classification_report(y_test, y_pred))
    print(f"Accuracy: {accuracy_score(y_test, y_pred)}")


Результати для Логістична регресія:
              precision    recall  f1-score   support

       False       0.89      0.99      0.94       344
        True       0.88      0.34      0.49        65

    accuracy                           0.89       409
   macro avg       0.88      0.66      0.71       409
weighted avg       0.89      0.89      0.87       409

Accuracy: 0.8875305623471883

Результати для Випадковий ліс:
              precision    recall  f1-score   support

       False       0.99      0.97      0.98       344
        True       0.85      0.92      0.88        65

    accuracy                           0.96       409
   macro avg       0.92      0.95      0.93       409
weighted avg       0.96      0.96      0.96       409

Accuracy: 0.960880195599022

Результати для Градієнтний бустинг:
              precision    recall  f1-score   support

       False       0.97      0.98      0.98       344
        True       0.89      0.85      0.87        65

    accuracy       